In [12]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.pyplot import figure
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
import datetime
from pdf2image import convert_from_path
import pandas as pd
from tqdm import tqdm_notebook

In [13]:
# This function will be used next to compute differences between the body sides
def compute_difference(smooth_image,x_sym,ymin,ymax,xmin,xmax):
    average_difference=0
    for i in range(ymin,ymax):
        k=xmin
        sum_difs=0
        sum_derivates=0
        while (smooth_image[i,x_sym-k]!=0 and smooth_image[i,x_sym+k]!=0 and k<xmax):
            #We calculate the sum of the difference of the derivatives on each side
            sum_difs+=smooth_image[i,x_sym+k]-smooth_image[i,x_sym-k]
            sum_derivates+=smooth_image[i,x_sym+k]-smooth_image[i,x_sym+k-1]-smooth_image[i,x_sym-k]+smooth_image[i,x_sym-k+1]
            k+=1
        if k!=1:
            average_difference+=(sum_difs+sum_derivates)/(k-1)
    return(average_difference/(ymax-ymin))

#This is the main function to generate the features
def image_features_generator(id_exam,path,X):
    
    #open the pdf as an image
    pages = convert_from_path(path,285)

    #We save the path of the pdf
    X.loc[id_exam, 'path'] = path
    
    # We read the exam date
    exam_date=datetime.date(1800,1,1)
    for i in range(len(path)-7):
        if path[i:i+8].isdecimal():
            temp_date=datetime.date(int(path[i:i+4]),int(path[i+4:i+6]),int(path[i+6:i+8]))
            if temp_date>exam_date:
                exam_date=temp_date
    X.loc[id_exam, 'exam_date'] = exam_date
    
    # We crop the colored level image zone
        # Define box inside image
    left = 125
    top = 1380
    width = 640
    height = 750
        # Create Box
    box = (left, top, left+width, top+height)
        # Crop Image
    area = pages[0].crop(box)
    np_im = np.array(area)
    
    #We compute the dominant color for each pixel (can be improved later)
    dominant_color=np.zeros((height, width), dtype=np.int)
        #we put value 4 when image is white
    dominant_color=dominant_color+4
    for i in range(height):
        for j in range(width):
            if np.sum(np_im[i,j,:])!=(3*255):
                dominant_color[i,j]=np.argmax(np_im[i,j,:])

    #Now we estimate the relative level between each points of the image
    level_image=np.zeros((height,width), dtype=np.int)
    token=100
    x_initiation=int(width/2)
    #First we fill the central column
    for i in range(height-1):
        if dominant_color[i+1,x_initiation]!=4:
            if dominant_color[i,x_initiation]!=4:
                if dominant_color[i+1,x_initiation]-dominant_color[i,x_initiation]==1:
                    token+=1
                elif dominant_color[i+1,x_initiation]-dominant_color[i,x_initiation]==-2:
                    token+=1
                elif dominant_color[i+1,x_initiation]-dominant_color[i,x_initiation]==-1:
                    token-=1
                elif dominant_color[i+1,x_initiation]-dominant_color[i,x_initiation]==2:
                    token-=1
            level_image[i+1,x_initiation]=token
    #Then we fill the right side
    for i in range(dominant_color.shape[0]):
        token=level_image[i,x_initiation]
        if token!=0:
            for j in range(dominant_color.shape[1]):
                    if j<x_initiation:
                        k=x_initiation-j-1
                        if dominant_color[i,k]!=4:
                            if dominant_color[i,k]-dominant_color[i,k+1]==1:
                                token+=1
                            elif dominant_color[i,k]-dominant_color[i,k+1]==-2:
                                token+=1
                            elif dominant_color[i,k]-dominant_color[i,k+1]==-1:
                                token-=1
                            elif dominant_color[i,k]-dominant_color[i,k+1]==2:
                                    token-=1  
                            level_image[i,k]=token
                    elif j==x_initiation:
                        token=level_image[i,x_initiation]
                    else:
                        if dominant_color[i,j]!=4:
                            if dominant_color[i,j]-dominant_color[i,j-1]==1:
                                token+=1
                            elif dominant_color[i,j]-dominant_color[i,j-1]==-2:
                                token+=1
                            elif dominant_color[i,j]-dominant_color[i,j-1]==-1:
                                token-=1
                            elif dominant_color[i,j]-dominant_color[i,j-1]==2:
                                    token-=1
                            level_image[i,j]=token

    #we apply a filter to the image to avoid a stairs effect
    smooth_image = cv2.blur(level_image,(5,5))
    mini=min(level_image[level_image>0])
    for i in range(height):
        for j in range(width):
            smooth_image[i,j] = smooth_image[i,j] if smooth_image[i,j]>=mini else 0
    
    #And now we seek for the symetry axis
    y=range(height)
    x= np.zeros(height, dtype=np.int)
    lower_mean_difs=2*(255**2)*height #this is the max possible for this number, so it can only go down
    x_sym=0
    ecart_sym=0
    for ecart in range(int(width/4)):
        #We do in one side
        j=int(width/2)-ecart
        mean_difs=0
        for i in y:
            k=1
            sum_difs=0
            sum_derivates=0
            while (smooth_image[i,j-k]!=0 and smooth_image[i,j+k]!=0):
                #We calculate the squared sum of the difference of the derivatives on each side
                sum_difs+=(smooth_image[i,j-k]-smooth_image[i,j+k])**2
                sum_derivates+=(smooth_image[i,j-k]-smooth_image[i,j-k+1]-smooth_image[i,j+k]+smooth_image[i,j+k-1])**2
                k+=1
            if k!=1:
                mean_difs+=(sum_difs+sum_derivates)/(k-1)
        if mean_difs < lower_mean_difs:
            lower_mean_difs=mean_difs
            x_sym=j
            ecart_sym=ecart
        #and we do the same in the other side
        j=int(width/2)+ecart
        mean_difs=0
        for i in y:
            k=1
            sum_difs=0
            sum_derivates=0
            while (smooth_image[i,j-k]!=0 and smooth_image[i,j+k]!=0):
                #We calculate the squared sum of the difference of the derivatives on each side
                sum_difs+=(smooth_image[i,j-k]-smooth_image[i,j+k])**2
                sum_derivates+=(smooth_image[i,j-k]-smooth_image[i,j-k+1]-smooth_image[i,j+k]+smooth_image[i,j+k-1])**2
                k+=1
            if k!=1:
                mean_difs+=(sum_difs+sum_derivates)/(k-1)
        if mean_difs < lower_mean_difs:
            lower_mean_difs=mean_difs
            x_sym=j
            ecart_sym=ecart
        #If we didn't update the symetry axis for a long time...we quit. It will significantly reduce the computing time
        if ecart-ecart_sym>20:
            break
    x.fill(x_sym)
    X.loc[id_exam, 'symetry_axis'] = x_sym
    
    #We measure some important values on the image
    z_left_scapula=smooth_image[:int(height/2),:x_sym].max()
    points=np.where(smooth_image[:int(height/2),:x_sym] == z_left_scapula)
    y_left_scapula=int(round(points[0].mean()))
    x_left_scapula=int(round(points[1].mean()))
    z_right_scapula=smooth_image[:int(height/2),x_sym:].max()
    points=np.where(smooth_image[:int(height/2),x_sym:] == z_right_scapula)
    y_right_scapula=int(round(points[0].mean()))
    x_right_scapula=int(round(points[1].mean()))+x_sym
    y_center_scapula=int((y_right_scapula+y_left_scapula)/2)
    x_center_scapula=int((x_right_scapula+x_left_scapula)/2)
    z_center_scapula=smooth_image[y_center_scapula,x_center_scapula]
    scapula_gap=x_right_scapula-x_left_scapula
    for i in range(smooth_image.shape[0]):
        nb_pts=np.count_nonzero(smooth_image[height-i-1,:])
        if nb_pts>scapula_gap:
            y_bottom=height-i-1
            break
    for i in range(height):
        nb_pts=np.count_nonzero(smooth_image[i,:])
        if nb_pts>scapula_gap:
            y_shoulders=i
            break               
    z_creux_du_dos=smooth_image[int(height/2):y_bottom,x_sym].min()
    points=np.where(smooth_image[int(height/2):y_bottom,x_sym] == z_creux_du_dos)
    y_creux_du_dos=int(round(points[0].mean())+height/2)

    #We define the interest zone (check if there are good with Manon... if not we should modify)
    y_thoracic_limit=y_center_scapula
    y_lombar_limit=y_creux_du_dos

    # Now we compute the interesting values we'll keep as features
    X.loc[id_exam, 'x_difference_scapulas'] = x_right_scapula-x_left_scapula
    X.loc[id_exam, 'x_offset_scapulas'] = x_sym-x_center_scapula
    X.loc[id_exam, 'y_difference_scapulas'] = y_right_scapula-y_left_scapula
    X.loc[id_exam, 'z_difference_scapulas'] = z_right_scapula-z_left_scapula
    X.loc[id_exam, 'y_scapulas_creux_du_dos'] = y_creux_du_dos-y_center_scapula
    X.loc[id_exam, 'z_scapulas_creux_du_dos'] = z_creux_du_dos-z_center_scapula
    X.loc[id_exam, 'average_dif_l_r_thoracic_close'] = compute_difference(smooth_image,x_sym,y_shoulders,y_thoracic_limit,1,int(scapula_gap/2))
    X.loc[id_exam, 'average_dif_l_r_thoracolombar_close'] = compute_difference(smooth_image,x_sym,y_thoracic_limit,y_lombar_limit,1,int(scapula_gap/2))
    X.loc[id_exam, 'average_dif_l_r_lombar_close'] = compute_difference(smooth_image,x_sym,y_lombar_limit,y_bottom,1,int(scapula_gap/2))
    X.loc[id_exam, 'average_dif_l_r_thoracic_far'] = compute_difference(smooth_image,x_sym,y_shoulders,y_thoracic_limit,int(scapula_gap/2),int(scapula_gap*0.75))
    X.loc[id_exam, 'average_dif_l_r_thoracolombar_far'] = compute_difference(smooth_image,x_sym,y_thoracic_limit,y_lombar_limit,int(scapula_gap/2),int(scapula_gap*0.75))
    X.loc[id_exam, 'average_dif_l_r_lombar_far'] = compute_difference(smooth_image,x_sym,y_lombar_limit,y_bottom,int(scapula_gap/2),int(scapula_gap*0.75))

In [14]:
#Generation of the exams features

#Declare the file
exams_X = pd.DataFrame(dtype=np.float64)

#For each file, we compute the features
id_exam=0
path1='../Florent/Documents/MANON/biomod PATIENTS/'
list_patients=os.listdir(path1)
for patient in tqdm_notebook(list_patients):
    if patient != 'Thumbs.db':
        id_patient=int(''.join(filter(str.isdecimal, patient)))
        list_exams=os.listdir(path1+patient)
        for exam in list_exams:
            if exam[-3:]=='pdf':
                path=path1+patient+'/'+exam
                image_features_generator(id_exam,path,exams_X)
                exams_X.loc[id_exam, 'id_patient'] = id_patient
                id_exam+=1
    

In [15]:
# Save the data and avoid computing again and again...
exams_X.to_csv('exams_X.csv', index=True)